In [3]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re
from collections import Counter



Импорт моделечки)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

Всё дальше пойдёт без дата сета, то как я вижу, что нужно сделать и как обучить модель, потому что до сих пор не смог вытащить этот дата сет с идиотской кегли блин(Как загружу, будут данные, пока покажу только алгоритм)
    

In [ ]:
data = pd.read_csv('IMDB Dataset.csv')
data.head()

In [ ]:
data.shape

In [ ]:
def no_of_words(text):
    words= text.split()
    word_count = len(words)
    return word_count

In [ ]:
data['word count'] = data['review'].apply(no_of_words)

In [ ]:
data.sentiment.replace("positive", 1, inplace=True)
data.sentiment.replace("negative", 2, inplace=True)

In [ ]:
def data_processing(text):
    text= text.lower()
    text = re.sub('<br />', '', text)
    text = re.sub(r"https\S+|www\S+|http\S+", '', text, flags = re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

In [ ]:
data.review = data['review'].apply(data_processing)

In [ ]:
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

In [ ]:
data.review = data['review'].apply(lambda x: stemming(x))

In [ ]:
pos_reviews = data[data.sentiment == 1]

In [ ]:
count = Counter()
for text in pos_reviews['review'].values:
    for word in text.split():
        count[word] +=1
count.most_common(15)

In [ ]:
pos_words = pd.DataFrame(count.most_common(15))
pos_words.columns = ['word', 'count']


In [ ]:
neg_reviews = data[data.sentiment == 2]

In [ ]:
count = Counter()
for text in neg_reviews['review'].values:
    for word in text.split():
        count[word] +=1
count.most_common(15)

In [ ]:
neg_words = pd.DataFrame(count.most_common(15))
neg_words.columns = ['word', 'count']


In [ ]:
X = data['review']
Y = data['sentiment']

In [ ]:
vector = TfidfVectorizer()
X = vector.fit_transform(data['review'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = .3)


In [ ]:
log_regression = LogisticRegression()
log_regression.fit(x_train, y_train)
log_predict = log_regression.predict(x_test)
log_accuracy = accuracy_score(log_predict, y_test)
print("Итак модель акинатор на: {:.1%}".format(log_accuracy * 100))